In [1]:
pip install -U langchain-huggingface


Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaLLM
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [23]:
# Load your local document
loader = TextLoader(r"AI_Full_Course_Overview.txt")
docs = loader.load()

# Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


In [46]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(splits, embedding=embeddings)
retriever = vectorstore.as_retriever()


In [24]:
import langchain_ollama
dir(langchain_ollama)



['ChatOllama',
 'OllamaEmbeddings',
 'OllamaLLM',
 'PackageNotFoundError',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_compat',
 '_raise_package_not_found_error',
 '_utils',
 'chat_models',
 'embeddings',
 'llms']

In [47]:
from langchain_ollama import ChatOllama



In [81]:
llm = ChatOllama(model="gemma3:270m")
# embeddings = OllamaEmbeddings(model="gemma3:latest")

In [82]:
from langchain_core.prompts import ChatPromptTemplate



In [83]:
prompt = ChatPromptTemplate.from_template("""
Use the following context to answer the question:
{context}

Question: {question}
""")


In [84]:
rag_chain = (
    {"context": retriever | (lambda docs: "\n\n".join([d.page_content for d in docs])),
     "question": RunnablePassthrough()}
    
    | prompt
    | llm
    | StrOutputParser()
)


In [85]:
query = "What are the key differences between Machine Learning and Deep Learning?"
response = rag_chain.invoke(query)
print(response)

The key differences between Machine Learning and Deep Learning lie in their approaches to building and training models, the types of models they are designed to handle, and the types of tasks they are best suited for.

*   **Machine Learning:** Focuses on learning patterns and making predictions from data without explicit programming. It involves algorithms that learn from data and make decisions based on those learned patterns.
*   **Deep Learning:** Employs artificial neural networks with multiple layers to learn complex representations of data. It is designed to handle large amounts of data and complex tasks with high accuracy.


In [86]:
query = "Explain what topics are covered in the Machine Learning section."
response = rag_chain.invoke(query)
print(response)

The Machine Learning section covers topics such as:

*   **Programming:** Python, R, TensorFlow, Keras, PyTorch, Scikit-learn, and Pandas.
*   **ML/DL:**  The core concepts of supervised, unsupervised, and reinforcement learning, algorithms like Linear Regression, Decision Trees, Random Forest, SVM, KNN, and their applications.
*   **Data Visualization & Analysis:** Creating visual insights with Matplotlib, Seaborn, and Plotly.
*   **Speech Recognition:** Converting spoken words to text using AI models.
*   **AI Ethics:**  Responsible AI usage, bias mitigation, transparency, and privacy.
*   **Tools & Frameworks:**  TensorFlow, Keras, PyTorch, Scikit-learn, and NumPy.
*   **Data Handling & ETL:** Moving data from raw sources to usable formats.
*   **Data Visualization & Analysis:** Creating visual insights with Matplotlib, Seaborn, and Plotly.

Therefore, the topics covered in the Machine Learning section are: **Programming, ML/DL, Data Visualization & Analysis, Speech Recognition, AI 